In [ ]:
import geopandas as gpd
import pandas as pd

In [ ]:
clusters = gpd.read_parquet("clusters_freiburg_ward_300.parquet").reset_index()

In [ ]:
clusters

In [ ]:
clusters.explore(column=clusters.index, cmap="tab20")

In [ ]:
data = pd.read_parquet("/data/processed_data/Bevoelkerung100M.parquet").reset_index()

In [ ]:
data

In [ ]:
data = data.rename(columns={"": "ID"})

In [ ]:
data

In [ ]:
gdf = gpd.GeoDataFrame(
    data,
    geometry=gpd.points_from_xy(data[("ID", "E")], data[("ID", "N")]),
    crs="EPSG:3035",
)
gdf = gdf.rename(columns={"": "geometry"})
gdf = gdf.drop(["N", "E"], axis=1, level=1)

In [ ]:
gdf

In [ ]:
# Buffer the points using a square cap style
# Note cap_style: round = 1, flat = 2, square = 3
gdf["geometry"] = gdf["geometry"].buffer(50, cap_style=3)

In [ ]:
gdf

In [ ]:
ntn = gdf[["NATIONALITY", "geometry"]]
ntn.columns = ntn[["NATIONALITY", "geometry"]].columns.get_level_values(1)

In [ ]:
ntn

In [ ]:
m = ntn.cx[4150000:4170000, 2760000:2780000].explore(column="Germany")
clusters.explore(m=m, column=clusters.index, cmap="tab20")

determine which cluster each 100m grid cell is in. choose the cluster covering the largest proportion of the cell

In [ ]:
cells = gdf[["ID", "geometry"]]

In [ ]:
cells.columns.get_level_values(0)

In [ ]:
cells.columns = cells.columns.get_level_values(0)

In [ ]:
cells

In [ ]:
clusters

In [ ]:
# Overlay cells with clusters to get intersections
overlap = gpd.overlay(cells, clusters, how="intersection")

# Calculate the area of overlap
overlap["area"] = overlap.geometry.area

In [ ]:
overlap.columns

In [ ]:
overlap

In [ ]:
overlap.explore(column="label")

In [ ]:
# Find the cluster with the largest overlap for each cell
largest_overlap = overlap.loc[overlap.groupby(overlap["ID"])["area"].idxmax()]

In [ ]:
largest_overlap

In [ ]:
# Merge this back with the original cells DataFrame to attach cluster information
result = cells.merge(largest_overlap[["ID", "label"]], on="ID")

In [ ]:
result

In [ ]:
result.explore("label")

In [ ]:
data["cluster", "cluster"] = data.droplevel(0, axis=1).merge(
    largest_overlap[["ID", "label"]], on="ID"
)["label"]

In [ ]:
data

In [ ]:
data

In [ ]:
data.to_parquet("/data/processed_data/Bevoelkerung100M_clusters.parquet")

In [ ]:
data[[" INSGESAMT", "NATIONALITY"]].fillna(0)

In [ ]:
data["STATS", "% German"] = (
    data["NATIONALITY", "Germany"].fillna(0) / data[" INSGESAMT", "Total"]
)

In [ ]:
data

In [ ]:
data[data["cluster", "cluster"] == 2.0]["STATS", "% German"].mean()

In [ ]:
for i in range(0, int(data["cluster", "cluster"].max())):
    print(data[data["cluster", "cluster"] == i]["STATS", "% German"].mean())